pre-trained: https://github.com/CompVis/stable-diffusion

In [1]:
# 필요한 라이브러리 설치
!pip install accelerate
!pip install diffusers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import random
# 이미지가 있는 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real'

# 디렉토리 내의 모든 파일 목록 가져오기
all_files = []
for f in os.listdir(directory_path):
  full_path = os.path.join(directory_path, f)
  if os.path.isfile(full_path):
    all_files.append(full_path)

# 파일 목록에서 무작위로 선택
random.seed(42)

if len(all_files) > 10:
    selected_files = random.sample(all_files, 10)
else:
    selected_files = all_files

In [5]:
print(selected_files)

['/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/62372.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/69496.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/10875.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/45903.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/49131.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/51815.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/59690.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/10630.jpg', '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/61888.jpg']


In [6]:
selected_files[0]

'/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg'

##### 생성된 결과가 train image와 너무 안 비슷하다. self-evolving의 개념을 활용해서 원본 이미지의 스타일과 최대한 비슷하면서도 prompt대로 따라갈 수는 없을까?

##### VGG-19 말고 Inception으로도 해보기

In [18]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# VGG19 네트워크 로드
class VGGFeatures(nn.Module):
    def __init__(self):
        super(VGGFeatures, self).__init__()
        vgg_pretrained_features = models.vgg19(pretrained=True).features

        self.layers = {
            '0': 'conv1_1',
            '5': 'conv2_1',
            '10': 'conv3_1',
            '19': 'conv4_1',
            '28': 'conv5_1'
        }

        self.features = nn.ModuleDict()
        for idx, layer_name in self.layers.items():
            self.features[layer_name] = vgg_pretrained_features[int(idx)]

        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        results = {}
        for layer_name, layer in self.features.items():
            x = layer(x)
            results[layer_name] = x
        return results


# 인스턴스화
vgg = VGGFeatures().to('cuda')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 152MB/s]


In [19]:
import torch.nn as nn
import torchvision.models as models

class InceptionFeatures(nn.Module):
    def __init__(self):
        super(InceptionFeatures, self).__init__()
        inception_pretrained_features = models.inception_v3(pretrained=True)

        # Inception v3는 aux_logits와 transform_input 등을 포함하므로 이를 조정
        self.inception = inception_pretrained_features
        self.inception.aux_logits = False
        self.inception.transform_input = False

        self.layers = {
            'Mixed_5b': 'mixed_5b',
            'Mixed_6a': 'mixed_6a',
            'Mixed_6b': 'mixed_6b',
            'Mixed_6c': 'mixed_6c',
            'Mixed_7a': 'mixed_7a'
        }

        self.features = nn.ModuleDict()
        for layer_name in self.layers.keys():
            self.features[layer_name] = getattr(self.inception, layer_name)

        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        results = {}
        # inception 모델의 preprocess 필요
        x = self.inception.Conv2d_1a_3x3(x)
        x = self.inception.Conv2d_2a_3x3(x)
        x = self.inception.Conv2d_2b_3x3(x)
        x = self.inception.maxpool1(x)
        x = self.inception.Conv2d_3b_1x1(x)
        x = self.inception.Conv2d_4a_3x3(x)
        x = self.inception.maxpool2(x)

        # 정의된 층에서 특징 추출
        for layer_name, layer in self.features.items():
            x = layer(x)
            results[layer_name] = x

        return results

# 인스턴스화
Inception = InceptionFeatures().to('cuda')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 165MB/s] 


In [20]:
# 이미지 전처리
def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    return preprocess(image) .unsqueeze(0)

def preprocess_image_wo_path(image):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# 스타일 특징 비교
def gram_matrix(input):
    a, b, c, d = input.size()
    features = input.view(a * b, c * d)
    G = torch.mm(features, features.t())
    return G.div(a * b * c * d)

def style_distance(target_features, style_features):
    distance = 0
    for layer in target_features.keys():
        gram_target = gram_matrix(target_features[layer])
        gram_style = gram_matrix(style_features[layer])
        distance += torch.nn.functional.l1_loss(gram_target, gram_style)
    return distance.item()

In [23]:
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import torch


# Stable Diffusion 모델 파이프라인을 초기화.
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    variant="fp16",
    torch_dtype=torch.float16,
    safety_checker = None
).to("cuda")

gender_list = ['male', 'female']
race_list = ['white', 'yellow', 'black']
age_list = ['baby', 'child', 'young-adult', 'middle-aged adult', 'old adult']

negative_prompt = ['Graffiti or Exaggeration or Unrealistic black and white image for Multiple people']

for i in range(10):
# Real face 이미지를 로드
  input_image_path = selected_files[i]
  img = Image.open(input_image_path).convert('RGB')
  #img = preprocess_image(input_image_path)

  # 이미지 생성
  gender = random.choice(gender_list)
  race = random.choice(race_list)
  age = random.choice(age_list)

  # 이미지 생성
  current_prompt = [f'A realistic photo of a {race} {gender}\'s face in {age}']

  generated_image = pipe(prompt = current_prompt, image=img, num_inference_steps=50, strength=0.75, guidance_scale=7.5, negative_prompt = negative_prompt).images[0]
  # strength: 1에 가까울 수록 원본 이미지에 가깝고, 0에 가까울 수록 prompt에 충실
  # guidance_scale: prompt를 얼마나 엄격하게 따를 것인가 7.5~20

  target_img = preprocess_image(input_image_path).to('cuda')
  gen_img = preprocess_image_wo_path(generated_image).to('cuda')

  target_feat = vgg(target_img)
  gen_feat = vgg(gen_img)

  distance = style_distance(target_feat, gen_feat)

  print(f'{i}-th image\'s Difference(distance): {distance}')

  threshold = 0.2

  if distance > threshold:
    add_prompt = ['being similar to the initial image']
    current_prompt = [''.join(current_prompt + add_prompt)]

    for j in range(5):
      generated_img = pipe(prompt = current_prompt, image = img, num_inference_steps = 50, strength = 0.75, guidance_scale = 7.5, negative_prompt = negative_prompt).images[0]

      gen_img = preprocess_image_wo_path(generated_img).to('cuda')
      gen_feat = vgg(gen_img)
      distance = style_distance(target_feat, gen_feat)

      if distance <= threshold:
          break

  path = f'/content/drive/MyDrive/ArtificialIntelligence/Project/GeneratedImages/image{i}.jpg'
  generated_img.save(path)

print("Completed generating images.")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/37 [00:00<?, ?it/s]

0-th image's Difference(distance): 0.2949574589729309


  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Completed generating images.


##### 비슷하게 만들 수 없다면, 처음부터 얼굴 부분만 segmentation해서 변형시킬 수는 없을까?

In [7]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=0a0e5389dce8f0d06f0160ee1906c4ff36599126bc9a1065aac8076ad1d0171e
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [17]:
import torch
import numpy as np
from PIL import Image
import face_recognition
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import pipeline
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

# 이미지 전처리를 위한 헬퍼 함수
def preprocess_image(image, target_size=(512, 512)):
    transform = Compose([
        Resize(target_size),  # 이미지 크기 조정
        ToTensor(),  # PIL 이미지를 텐서로 변환
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 이미지 정규화
    ])
    return transform(image).unsqueeze(0)  # 배치 차원 추가

def modify_face_attributes(image_path, prompt, model_id="CompVis/stable-diffusion-v1-4"):
    # 모델 로드
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker = None).to(device)

    # 이미지 로드 및 얼굴 인식
    #image = face_recognition.load_image_file(image_path)
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)
    face_locations = face_recognition.face_locations(image_np)

    if not face_locations:
        print("No faces were found in the image.")
        return

    # 첫 번째 얼굴 사용
    top, right, bottom, left = face_locations[0]
    face_image_np = image_np[top:bottom, left:right]
    face_image = Image.fromarray(face_image_np)  # numpy 배열을 PIL 이미지로 변환
    #processed_face_image = preprocess_image(face_image)  # 이미지 전처리

    # 이미지-투-이미지 변환 실행
    modified_face = pipe(prompt=prompt, image=face_image, num_inference_steps=50, strength=0.75, guidance_scale=7.5).images[0]

    # 원본 이미지에 얼굴 합성
    original_image = Image.open(image_path)
    original_image.paste(modified_face, (left, top))

    # 결과 이미지 저장 또는 표시
    modified_image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/GeneratedImages/image.jpg"
    original_image.save(modified_image_path)
    original_image.show()

    return modified_image_path

# 사용 예제
image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg"
prompt = "A young Asian female face, realistic style"
modified_image_path = modify_face_attributes(image_path, prompt)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/37 [00:00<?, ?it/s]